In [ ]:
import sys
sys.path.append("..")
from importlib import import_module
import torch,time,os,random
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append("..")
import LIAF
from datasets.cifar10_dvs_large import cifar10_DVS
from tensorboardX import SummaryWriter
from importlib import import_module
from LIAFnet.LIAFResNet import *
autocast = LIAF.autocast #统一autocast模式

################################ parameters ####################
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
for test in range(0,10):
    task = 'Large_with_pretarin' + str(test)
    writer = SummaryWriter(comment=task)
    learning_rate = 1e-3
    batch_size  = 64
    num_epochs = 100
    timeWindows = 8
    TA.timeWindows = 8 #用了attention必须固定时间长度
    
    names = 'dvs_cifar10'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_path = '/data/CIFAR10-MAT' #r := raw string
    train_dataset = cifar10_DVS(train_path,'train',timeWindows) #max= 25?
    test_dataset = cifar10_DVS(train_path,'test',timeWindows)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=batch_size,
                                               shuffle=False)

    TA.timeWindows = 8 #用了attention必须固定时间长度
    modules = import_module('LIAFnet.LIAFResNet_18')
    config  = modules.Config()
    workpath = os.path.abspath(os.getcwd())
    num_epochs = 60
    accumulation = config.accumulation
    timeWindows = config.timeWindows
    config.cfgCnn = [2, 64, 7, True]
    config.dataSize = (128,128)
    config.actFun= LIAF.LIFactFun.apply
    epoch = 0
    bestacc = 0  # best test accuracy
    start_epoch = 0  # start from epoch 0 or last checkpoint epoch
    training_iter = 0
    snn = LIAFResNet(config)
    checkpoint = torch.load('./45.pkl', map_location=torch.device('cpu'))
    snn.load_state_dict(checkpoint)
    
    for p in snn.parameters():
        p.requires_grad=False #TODO
    snn.fc = nn.Linear(snn.cfgFc_[0],10)
    print("Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in snn.parameters())))

    snn.to(device)
    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(snn.parameters(),
                    lr=learning_rate,
                    weight_decay =1e-4)


    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
                        optimizer, 
                        milestones=[30,40,50], 
                        gamma=0.1, 
                        last_epoch=-1)
    def val(optimizer,snn,test_loader,test_dataset,batch_size,epoch):
        print('===> evaluating models...')
        snn.eval()
        correct = 0
        total = 0
        predicted = 0
        with torch.no_grad():
            for batch_idx, (inputs, labels) in enumerate(test_loader):
                if ((batch_idx+1)<=len(test_dataset)//batch_size):
                    optimizer.zero_grad()
                    outputs = snn(inputs.to(device))
                    labels = labels.squeeze(1)
                    _ , predicted = outputs.cpu().max(1)
                    total += float(labels.size(0))
                    correct += float(predicted.eq(labels).sum())
        acc = 100. * float(correct) / float(total)
        print('================')
        print('val acc:',acc , '%  epoch:',epoch)
        print('================')
        writer.add_scalar('acc', acc, epoch)
        return acc

    save_folder = './saved_model'

    for epoch in range(num_epochs):
        # 新增2：设置sampler的epoch，DistributedSampler需要这个来维持各个进程之间的相同随机数种子
        #train_loader.sampler.set_epoch(epoch)
        if epoch>1:
            for p in snn.parameters():
                p.requires_grad=True #TODO
        snn.train()
        running_loss = 0
        start_time = time.time() 
        print('===> training models...')
        correct = 0.0
        total = 0.0
        torch.cuda.empty_cache()
        for i, (images, labels) in enumerate(train_loader):
            if ((i+1)<=len(train_dataset)//batch_size):
                with autocast():
                    outputs = snn(images.to(device)).cpu()
                    labels = labels.squeeze(1)
                    _ , predict = outputs.max(1)
                    loss = criterion(outputs, labels)
                    correct += predict.eq(labels).sum()
                    total += float(predict.size(0))

                    loss /= accumulation
                    running_loss += loss.item()
                    loss.backward()

                if (i+1)%accumulation == 0:
                    optimizer.step()
                    snn.zero_grad()
                    optimizer.zero_grad()

                if (i+1)%(10) == 0:
                    if not os.path.isdir(save_folder):
                        os.mkdir(save_folder)
                    train_acc =  100* correct / total
                    print('=====> Epoch [%d/%d], Step [%d/%d], Loss: %.5f , Acc: %.5f  '
                      %(epoch+start_epoch, num_epochs+start_epoch, i+1, len(train_dataset)//(batch_size),running_loss, train_acc))
                    writer.add_scalar('running Loss', running_loss, training_iter)
                    writer.add_scalar('running Acc', train_acc, training_iter)
                    correct = 0.0
                    total = 0.0
                    running_loss = 0
                    #
            training_iter +=1 
        torch.cuda.empty_cache()
        #evaluation
        acc = val(optimizer,snn,test_loader,test_dataset,batch_size,epoch)
        lr_scheduler.step()
        if not os.path.isdir(save_folder):
            os.mkdir(save_folder)
        if acc > bestacc:
            bestacc = acc
            print('===> Saving models...')


    torch.save(snn.state_dict(),'./'+save_folder+'/'+task+str(int(bestacc))+'.pkl')

use torch.cuda.amp.autocast for fusion prcision training
Total number of paramerters in networks is 11185379  
===> training models...


/home/lyh/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


=====> Epoch [0/60], Step [10/140], Loss: 20.57126 , Acc: 24.37500  
=====> Epoch [0/60], Step [20/140], Loss: 17.76989 , Acc: 35.46875  
=====> Epoch [0/60], Step [30/140], Loss: 17.02118 , Acc: 37.81250  
=====> Epoch [0/60], Step [40/140], Loss: 16.27520 , Acc: 42.18750  
=====> Epoch [0/60], Step [50/140], Loss: 15.58616 , Acc: 43.43750  
=====> Epoch [0/60], Step [60/140], Loss: 14.84928 , Acc: 46.56250  
=====> Epoch [0/60], Step [70/140], Loss: 14.32769 , Acc: 49.53125  
=====> Epoch [0/60], Step [80/140], Loss: 14.51199 , Acc: 49.53125  
=====> Epoch [0/60], Step [90/140], Loss: 14.74642 , Acc: 48.28125  
=====> Epoch [0/60], Step [100/140], Loss: 13.50469 , Acc: 52.65625  
=====> Epoch [0/60], Step [110/140], Loss: 14.25400 , Acc: 50.93750  
=====> Epoch [0/60], Step [120/140], Loss: 12.71234 , Acc: 55.93750  
=====> Epoch [0/60], Step [130/140], Loss: 12.85976 , Acc: 54.84375  
=====> Epoch [0/60], Step [140/140], Loss: 13.59897 , Acc: 51.25000  
===> evaluating models...
val

=====> Epoch [8/60], Step [50/140], Loss: 5.98132 , Acc: 79.06250  
=====> Epoch [8/60], Step [60/140], Loss: 6.14948 , Acc: 79.53125  
=====> Epoch [8/60], Step [70/140], Loss: 5.74046 , Acc: 80.93750  
=====> Epoch [8/60], Step [80/140], Loss: 6.09406 , Acc: 79.21875  
=====> Epoch [8/60], Step [90/140], Loss: 6.94632 , Acc: 76.09375  
=====> Epoch [8/60], Step [100/140], Loss: 6.48710 , Acc: 78.43750  
=====> Epoch [8/60], Step [110/140], Loss: 6.96084 , Acc: 75.00000  
=====> Epoch [8/60], Step [120/140], Loss: 7.75871 , Acc: 73.90625  
=====> Epoch [8/60], Step [130/140], Loss: 7.02122 , Acc: 77.03125  
=====> Epoch [8/60], Step [140/140], Loss: 5.88104 , Acc: 79.68750  
===> evaluating models...
val acc: 38.958333333333336 %  epoch: 8
===> training models...
=====> Epoch [9/60], Step [10/140], Loss: 4.84505 , Acc: 84.06250  
=====> Epoch [9/60], Step [20/140], Loss: 5.03034 , Acc: 82.96875  
=====> Epoch [9/60], Step [30/140], Loss: 5.01456 , Acc: 82.65625  
=====> Epoch [9/60], 

=====> Epoch [15/60], Step [130/140], Loss: 3.17294 , Acc: 88.59375  
=====> Epoch [15/60], Step [140/140], Loss: 3.36323 , Acc: 87.96875  
===> evaluating models...
val acc: 54.0625 %  epoch: 15
===> Saving models...
===> training models...
=====> Epoch [16/60], Step [10/140], Loss: 1.86417 , Acc: 94.06250  
=====> Epoch [16/60], Step [20/140], Loss: 2.69176 , Acc: 90.62500  
=====> Epoch [16/60], Step [30/140], Loss: 2.72382 , Acc: 90.93750  
=====> Epoch [16/60], Step [40/140], Loss: 3.03129 , Acc: 89.06250  
=====> Epoch [16/60], Step [50/140], Loss: 2.68496 , Acc: 89.68750  
=====> Epoch [16/60], Step [60/140], Loss: 2.36161 , Acc: 92.65625  
=====> Epoch [16/60], Step [70/140], Loss: 2.93459 , Acc: 89.37500  
=====> Epoch [16/60], Step [80/140], Loss: 2.94199 , Acc: 89.21875  
=====> Epoch [16/60], Step [90/140], Loss: 3.49068 , Acc: 88.90625  
=====> Epoch [16/60], Step [100/140], Loss: 4.16180 , Acc: 86.40625  
=====> Epoch [16/60], Step [110/140], Loss: 3.32543 , Acc: 87.65625

=====> Epoch [23/60], Step [40/140], Loss: 1.48558 , Acc: 95.78125  
=====> Epoch [23/60], Step [50/140], Loss: 1.41054 , Acc: 96.09375  
=====> Epoch [23/60], Step [60/140], Loss: 1.28526 , Acc: 96.09375  
=====> Epoch [23/60], Step [70/140], Loss: 1.36480 , Acc: 95.15625  
=====> Epoch [23/60], Step [80/140], Loss: 1.50220 , Acc: 95.46875  
=====> Epoch [23/60], Step [90/140], Loss: 1.45852 , Acc: 94.68750  
=====> Epoch [23/60], Step [100/140], Loss: 1.51404 , Acc: 95.00000  
=====> Epoch [23/60], Step [110/140], Loss: 1.60310 , Acc: 94.84375  
=====> Epoch [23/60], Step [120/140], Loss: 1.61363 , Acc: 93.75000  
=====> Epoch [23/60], Step [130/140], Loss: 1.36316 , Acc: 95.00000  
=====> Epoch [23/60], Step [140/140], Loss: 1.76671 , Acc: 93.75000  
===> evaluating models...
val acc: 48.645833333333336 %  epoch: 23
===> training models...
=====> Epoch [24/60], Step [10/140], Loss: 1.28197 , Acc: 96.09375  
=====> Epoch [24/60], Step [20/140], Loss: 1.43781 , Acc: 95.62500  
=====> 

=====> Epoch [30/60], Step [130/140], Loss: 0.49367 , Acc: 98.90625  
=====> Epoch [30/60], Step [140/140], Loss: 0.37234 , Acc: 98.90625  
===> evaluating models...
val acc: 66.66666666666667 %  epoch: 30
===> Saving models...
===> training models...
=====> Epoch [31/60], Step [10/140], Loss: 0.30924 , Acc: 99.21875  
=====> Epoch [31/60], Step [20/140], Loss: 0.23725 , Acc: 99.53125  
=====> Epoch [31/60], Step [30/140], Loss: 0.28382 , Acc: 99.37500  
=====> Epoch [31/60], Step [40/140], Loss: 0.25535 , Acc: 99.53125  
=====> Epoch [31/60], Step [50/140], Loss: 0.25639 , Acc: 99.37500  
=====> Epoch [31/60], Step [60/140], Loss: 0.26249 , Acc: 99.21875  
=====> Epoch [31/60], Step [70/140], Loss: 0.19483 , Acc: 99.53125  
=====> Epoch [31/60], Step [80/140], Loss: 0.20432 , Acc: 99.53125  
=====> Epoch [31/60], Step [90/140], Loss: 0.22360 , Acc: 99.68750  
=====> Epoch [31/60], Step [100/140], Loss: 0.22262 , Acc: 99.53125  
=====> Epoch [31/60], Step [110/140], Loss: 0.20293 , Acc

=====> Epoch [38/60], Step [30/140], Loss: 0.06061 , Acc: 100.00000  
=====> Epoch [38/60], Step [40/140], Loss: 0.05199 , Acc: 100.00000  
=====> Epoch [38/60], Step [50/140], Loss: 0.05431 , Acc: 100.00000  
=====> Epoch [38/60], Step [60/140], Loss: 0.06441 , Acc: 99.84375  
=====> Epoch [38/60], Step [70/140], Loss: 0.05601 , Acc: 99.84375  
=====> Epoch [38/60], Step [80/140], Loss: 0.03256 , Acc: 100.00000  
=====> Epoch [38/60], Step [90/140], Loss: 0.05427 , Acc: 100.00000  
=====> Epoch [38/60], Step [100/140], Loss: 0.12965 , Acc: 99.84375  
=====> Epoch [38/60], Step [110/140], Loss: 0.08108 , Acc: 99.84375  
=====> Epoch [38/60], Step [120/140], Loss: 0.04541 , Acc: 100.00000  
=====> Epoch [38/60], Step [130/140], Loss: 0.06597 , Acc: 100.00000  
=====> Epoch [38/60], Step [140/140], Loss: 0.07515 , Acc: 99.68750  
===> evaluating models...
val acc: 67.39583333333333 %  epoch: 38
===> training models...
=====> Epoch [39/60], Step [10/140], Loss: 0.04470 , Acc: 100.00000  


In [ ]:
61.197916666666664 %  epoch: 14
63.28125 %  epoch: 31